# Pipeline de ETL utilizando a NASA Open API

## Introdução
Este projeto foi criado com o objetivo de construir uma pipeline de ETL (Extraction, Transformation and Loading) utilizando Python. Decidi trabalhar com as APIs da NASA, que além de serem totalmente abertas e fáceis de usar, são extremamente rica em informações.

Dentre as muitas APIs disponibilizadas pela NASA, me chamoaram atenção as do DONKI (The Space Weather Database Of Notifications, Knowledge, Information). O DONKI traz interpretações sobre observações climáticas espaciais, isto é, de fenômenos como explosões solares, tempestades geomagnéticas, ejeção de massa coronal, entre outros eventos meteorológicos do espaço.

Em particular, decidi estudar a correlação entre as ejeções de massa coronal (Coronal Mass Ejections - CMEs) e as explosões solares (Solar Flares). Vamos ao que são, em linhas gerais, esses dois fenômenos e como estão ligados:
* Uma explosão solar é causada pela liberação súbita de grandes quantidades de energia armazenada em campos magnéticos no sol
* Uma ejeção de massa coronal (ou CME) é uma nuvem gigante de plasma solar encharcada com linhas de campo magnético que é soprada para longe do Sol. Quando atingem a terra, provocam o que chamamos de tempestades geomagnéticas. Estão relacionadas às explosões solares pois podem ser ocasionadas por estas, pois a liberação da quantidade enorme de energia advinda de uma explosão solar pode perturbar o campo magnético do sol, facilitando as CMEs.
FONTE: https://www.spaceweatherlive.com/pt/ajuda/o-que-e-uma-ejecao-de-massa-coronal-emc.html

## APIs Utilizadas
Coronal Mass Ejection Analysis:   

    https://ccmc.gsfc.nasa.gov/tools/DONKI/#coronal-mass-ejection--cme--analysis

Solar Flare:

    https://ccmc.gsfc.nasa.gov/tools/DONKI/#solar-flare--flr-

Para visualizar no Postman, com explicações e exemplos de GET requests: https://www.postman.com/miguelolave/workspace/nasa-open-apis/collection/3419756-c07cfadb-8b45-4441-9483-95b61d99a1c6?action=share&creator=30604185

## Extração
Extraindo os dados das APIs.

In [429]:
import requests
from urllib.request import urlretrieve
from pprint import PrettyPrinter
pp = PrettyPrinter()

In [430]:
api_key = 'fpSjmq2bgox8ZUCskKmaFUHihezeVKr1S01jcX3T'

In [431]:
import requests
from pprint import PrettyPrinter

# GET request Solar Flare API
def fetch_flr():
    URL_FLR = "https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/FLR"

    # Parametros da query
    params = {
        'api_key': api_key,
        'startDate': '2020-01-01',
        'endDate': '2023-01-01',
    }

    response = requests.get(URL_FLR, params=params).json()
    #pp = PrettyPrinter()
    #pp.pprint(response) # Printar JSON

    return response

response_flr = fetch_flr()

In [432]:
flr_df = pd.DataFrame(response_flr)
flr_df[0:5]

,flrID,instruments,beginTime,peakTime,endTime,classType,sourceLocation,activeRegionNum,linkedEvents,link
0,2020-05-29T07:18:00-FLR-001,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2020-05-29T07:18Z,2020-05-29T07:54Z,None,M1.1,N32E90,NaN,None,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...
1,2020-11-20T16:44:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2020-11-20T16:44Z,2020-11-20T17:03Z,None,C1.9,S22E40,12783.0,[{'activityID': '2020-11-20T19:51:00-CME-001'}],https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...
2,2020-11-20T18:11:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2020-11-20T18:11Z,2020-11-20T18:41Z,None,C1.0,S25E120,NaN,None,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...
3,2020-11-29T12:34:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2020-11-29T12:34Z,2020-11-29T13:11Z,2020-11-29T13:41Z,M4.4,S25E97,NaN,[{'activityID': '2020-11-29T13:24:00-CME-001'}...,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...
4,2020-12-07T15:46:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2020-12-07T15:46Z,2020-12-07T16:32Z,None,C7.4,S23W11,12790.0,[{'activityID': '2020-12-07T16:24:00-CME-001'}],https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...


In [433]:
import requests
from pprint import PrettyPrinter

api_key = 'fpSjmq2bgox8ZUCskKmaFUHihezeVKr1S01jcX3T'

# Making a GET request to the CME API
def fetch_cme():
    URL_CME= "https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/CMEAnalysis"

    # Parametros da query
    params = {
        'api_key': api_key,
        'startDate': '2020-01-01',
        'endDate': '2023-01-01',
        'mostAccurateOnly': 'true',
        'completeEntryOnly': 'true',
        #'speed': '0',
        #'halfAngle':  '0'
    }

    response = requests.get(URL_CME, params=params).json()
    #pp = PrettyPrinter()
    #pp.pprint(response) # Printar JSON

    return response

response_cme = fetch_cme()

In [434]:
cme_df = pd.DataFrame(response_cme)
cme_df[0:5]

,time21_5,latitude,longitude,halfAngle,speed,type,isMostAccurate,associatedCMEID,note,catalog,link
0,2020-01-06T16:13Z,-2.0,9.0,19.0,227.0,S,True,2020-01-05T16:45:00-CME-001,,M2M_CATALOG,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...
1,2020-01-15T03:44Z,-5.0,12.0,6.0,205.0,S,True,2020-01-14T11:09:00-CME-001,Very uncertain measurement. There was only thr...,M2M_CATALOG,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...
2,2020-01-19T23:11Z,5.0,-115.0,17.0,193.0,S,True,2020-01-18T20:12:00-CME-001,The source is unclear.,M2M_CATALOG,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...
3,2020-01-26T04:27Z,2.0,-17.0,10.0,362.0,S,True,2020-01-25T18:54:00-CME-001,,M2M_CATALOG,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...
4,2020-02-12T08:51Z,2.0,-45.0,21.0,341.0,S,True,2020-02-11T17:41:00-CME-001,,M2M_CATALOG,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/CM...


## Transformação
Início do tratamento e transformação dos dados das APIs. A primeira etapa foi relacionar as CMEs às explosões solares causadas por elas (dado disponilizado pela própria NASA, chave 'linkedEvents' da API Solar Flare).

In [435]:
# Definindo quais CMEs estão ligadas a quais flares
linked_cmes = []
for item in response_flr:
    linked_events = item.get('linkedEvents')
    flr_id = item.get('flrID')
    if linked_events is not None:
        for index,event in enumerate(linked_events):
            activity_id = event.get('activityID')
            if activity_id[20:23] == 'CME': # Essa parte identifica qual o tipo de atividade causada, nesse caso queremos CMEs
                #print(f"Flare id: {flr_id}")
                #print(f"CME id: {activity_id}")
                linked_cmes.append((activity_id,flr_id))
#print(linked_cmes)

In [436]:
import pandas as pd
df_linked = pd.DataFrame(linked_cmes,columns=['cmes', 'flares']) # DataFrame contendo apenas os eventos interligados
df_linked[0:5]

,cmes,flares
0,2020-11-20T19:51:00-CME-001,2020-11-20T16:44:00-FLR-001
1,2020-11-29T13:24:00-CME-001,2020-11-29T12:34:00-FLR-001
2,2020-12-07T16:24:00-CME-001,2020-12-07T15:46:00-FLR-001
3,2020-12-13T09:24:00-CME-001,2020-12-13T08:45:00-FLR-001
4,2020-12-14T15:12:00-CME-001,2020-12-14T14:09:00-FLR-001


In [437]:
# Inserindo todas as colunas com dados para os CMEs e solar flares

for column_name in cme_df.columns:
  #print(column_name)
  df_linked.insert(loc=1, column=f'{column_name}', value=None)
#print(df_linked)
for column_name in flr_df.columns:
  #print(column_name)
  df_linked[f'{column_name}'] = None
df_linked[0:5]

,cmes,link,catalog,note,associatedCMEID,isMostAccurate,type,speed,halfAngle,longitude,...,flares,flrID,instruments,beginTime,peakTime,endTime,classType,sourceLocation,activeRegionNum,linkedEvents
0,2020-11-20T19:51:00-CME-001,None,None,None,None,None,None,None,None,None,...,2020-11-20T16:44:00-FLR-001,None,None,None,None,None,None,None,None,None
1,2020-11-29T13:24:00-CME-001,None,None,None,None,None,None,None,None,None,...,2020-11-29T12:34:00-FLR-001,None,None,None,None,None,None,None,None,None
2,2020-12-07T16:24:00-CME-001,None,None,None,None,None,None,None,None,None,...,2020-12-07T15:46:00-FLR-001,None,None,None,None,None,None,None,None,None
3,2020-12-13T09:24:00-CME-001,None,None,None,None,None,None,None,None,None,...,2020-12-13T08:45:00-FLR-001,None,None,None,None,None,None,None,None,None
4,2020-12-14T15:12:00-CME-001,None,None,None,None,None,None,None,None,None,...,2020-12-14T14:09:00-FLR-001,None,None,None,None,None,None,None,None,None


In [438]:
# Preenchendo os dados

# CMEs
for index, row in df_linked.iterrows():
  #print(index,row)
  #df_linked.loc[index]['cmes']
  for item_cme in response_cme:
    if item_cme['associatedCMEID'] == df_linked.loc[index]['cmes']:
      for column_name in cme_df.columns:
        df_linked.loc[index][f'{column_name}'] = item_cme[f'{column_name}']

# Solar Flares
for index, row in df_linked.iterrows():
  #print(index,row)
  #df_linked.loc[index]['cmes']
  for item_flr in response_flr:
    if item_flr['flrID'] == df_linked.loc[index]['flares']:
      for column_name in flr_df.columns:
        df_linked.loc[index][f'{column_name}'] = item_flr[f'{column_name}']

df_linked[0:5]

,cmes,link,catalog,note,associatedCMEID,isMostAccurate,type,speed,halfAngle,longitude,...,flares,flrID,instruments,beginTime,peakTime,endTime,classType,sourceLocation,activeRegionNum,linkedEvents
0,2020-11-20T19:51:00-CME-001,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,M2M_CATALOG,,2020-11-20T19:51:00-CME-001,True,O,1166.0,15.5,-39.0,...,2020-11-20T16:44:00-FLR-001,2020-11-20T16:44:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2020-11-20T16:44Z,2020-11-20T17:03Z,None,C1.9,S22E40,12783,[{'activityID': '2020-11-20T19:51:00-CME-001'}]
1,2020-11-29T13:24:00-CME-001,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,M2M_CATALOG,,2020-11-29T13:24:00-CME-001,True,O,1336.0,58.0,-75.0,...,2020-11-29T12:34:00-FLR-001,2020-11-29T12:34:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2020-11-29T12:34Z,2020-11-29T13:11Z,2020-11-29T13:41Z,M4.4,S25E97,None,[{'activityID': '2020-11-29T13:24:00-CME-001'}...
2,2020-12-07T16:24:00-CME-001,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,M2M_CATALOG,This is an updated measurement that uses a neg...,2020-12-07T16:24:00-CME-001,True,O,1383.0,41.0,12.0,...,2020-12-07T15:46:00-FLR-001,2020-12-07T15:46:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2020-12-07T15:46Z,2020-12-07T16:32Z,None,C7.4,S23W11,12790,[{'activityID': '2020-12-07T16:24:00-CME-001'}]
3,2020-12-13T09:24:00-CME-001,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,M2M_CATALOG,This CME analysis used SOHO LASCO C3 differenc...,2020-12-13T09:24:00-CME-001,True,C,610.0,26.0,-121.0,...,2020-12-13T08:45:00-FLR-001,2020-12-13T08:45:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2020-12-13T08:45Z,2020-12-13T08:55Z,2020-12-13T09:08Z,B1.4,S18E90,None,[{'activityID': '2020-12-13T09:24:00-CME-001'}]
4,2020-12-14T15:12:00-CME-001,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,M2M_CATALOG,,2020-12-14T15:12:00-CME-001,True,S,390.0,19.5,-55.0,...,2020-12-14T14:09:00-FLR-001,2020-12-14T14:09:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2020-12-14T14:09Z,2020-12-14T14:37Z,2020-12-14T14:56Z,C4.0,S22E44,12792,[{'activityID': '2020-12-14T15:12:00-CME-001'}]


In [439]:
# Deletando colunas que não iremos utilizar
cols_to_delete = ['beginTime','endTime','activeRegionNum','linkedEvents','link','catalog','associatedCMEID','isMostAccurate','instruments','flrID','note']
df_linked = df_linked.drop(columns=cols_to_delete, axis=1)
df_linked[0:5]

,cmes,type,speed,halfAngle,longitude,latitude,time21_5,flares,peakTime,classType,sourceLocation
0,2020-11-20T19:51:00-CME-001,O,1166.0,15.5,-39.0,3.0,2020-11-20T22:48Z,2020-11-20T16:44:00-FLR-001,2020-11-20T17:03Z,C1.9,S22E40
1,2020-11-29T13:24:00-CME-001,O,1336.0,58.0,-75.0,-6.0,2020-11-29T16:00Z,2020-11-29T12:34:00-FLR-001,2020-11-29T13:11Z,M4.4,S25E97
2,2020-12-07T16:24:00-CME-001,O,1383.0,41.0,12.0,-21.0,2020-12-07T18:49Z,2020-12-07T15:46:00-FLR-001,2020-12-07T16:32Z,C7.4,S23W11
3,2020-12-13T09:24:00-CME-001,C,610.0,26.0,-121.0,3.0,2020-12-13T14:41Z,2020-12-13T08:45:00-FLR-001,2020-12-13T08:55Z,B1.4,S18E90
4,2020-12-14T15:12:00-CME-001,S,390.0,19.5,-55.0,-13.2,2020-12-15T00:16Z,2020-12-14T14:09:00-FLR-001,2020-12-14T14:37Z,C4.0,S22E44


In [440]:
# Convertendo o formato de latitude e longitude das Solar Flares
df_linked['latitude_flr'] = None
df_linked['longitude_flr'] = None

for index, row in df_linked.iterrows():
    lat_dir = row['sourceLocation'][0]
    lon_dir = row['sourceLocation'][3]
    latitude = int(row['sourceLocation'][1:3])
    longitude = int(row['sourceLocation'][4:])

    # Dependendo da direção, serão valores positivos ou negativos
    if lat_dir == 'N':
        latitude = -latitude
    if lon_dir == 'W':
        longitude = -longitude

    df_linked.at[index, 'latitude_flr'] = latitude
    df_linked.at[index, 'longitude_flr'] = longitude

df_linked[0:5]

,cmes,type,speed,halfAngle,longitude,latitude,time21_5,flares,peakTime,classType,sourceLocation,latitude_flr,longitude_flr
0,2020-11-20T19:51:00-CME-001,O,1166.0,15.5,-39.0,3.0,2020-11-20T22:48Z,2020-11-20T16:44:00-FLR-001,2020-11-20T17:03Z,C1.9,S22E40,22,40
1,2020-11-29T13:24:00-CME-001,O,1336.0,58.0,-75.0,-6.0,2020-11-29T16:00Z,2020-11-29T12:34:00-FLR-001,2020-11-29T13:11Z,M4.4,S25E97,25,97
2,2020-12-07T16:24:00-CME-001,O,1383.0,41.0,12.0,-21.0,2020-12-07T18:49Z,2020-12-07T15:46:00-FLR-001,2020-12-07T16:32Z,C7.4,S23W11,23,-11
3,2020-12-13T09:24:00-CME-001,C,610.0,26.0,-121.0,3.0,2020-12-13T14:41Z,2020-12-13T08:45:00-FLR-001,2020-12-13T08:55Z,B1.4,S18E90,18,90
4,2020-12-14T15:12:00-CME-001,S,390.0,19.5,-55.0,-13.2,2020-12-15T00:16Z,2020-12-14T14:09:00-FLR-001,2020-12-14T14:37Z,C4.0,S22E44,22,44


In [441]:
# Renomeando algumas colunas
df_linked = df_linked.rename(columns={'latitude': 'latitude_cme', 'longitude': 'longitude_cme','time21_5':'time_cme'})

In [442]:
# Removendo a coluna sourceLocation, que não será mais utilizada
df_linked = df_linked.drop('sourceLocation', axis=1)

In [443]:
df_linked[0:5]

,cmes,type,speed,halfAngle,longitude_cme,latitude_cme,time_cme,flares,peakTime,classType,latitude_flr,longitude_flr
0,2020-11-20T19:51:00-CME-001,O,1166.0,15.5,-39.0,3.0,2020-11-20T22:48Z,2020-11-20T16:44:00-FLR-001,2020-11-20T17:03Z,C1.9,22,40
1,2020-11-29T13:24:00-CME-001,O,1336.0,58.0,-75.0,-6.0,2020-11-29T16:00Z,2020-11-29T12:34:00-FLR-001,2020-11-29T13:11Z,M4.4,25,97
2,2020-12-07T16:24:00-CME-001,O,1383.0,41.0,12.0,-21.0,2020-12-07T18:49Z,2020-12-07T15:46:00-FLR-001,2020-12-07T16:32Z,C7.4,23,-11
3,2020-12-13T09:24:00-CME-001,C,610.0,26.0,-121.0,3.0,2020-12-13T14:41Z,2020-12-13T08:45:00-FLR-001,2020-12-13T08:55Z,B1.4,18,90
4,2020-12-14T15:12:00-CME-001,S,390.0,19.5,-55.0,-13.2,2020-12-15T00:16Z,2020-12-14T14:09:00-FLR-001,2020-12-14T14:37Z,C4.0,22,44


In [444]:
df_final = df_linked.rename(columns={'cmes': 'cme_id', 'halfAngle': 'half_angle','flares':'flare_id','peakTime':'peak_time_flr','type':'type_cme','classType':'type_flr'})
df_final[0:5]

,cme_id,type_cme,speed,half_angle,longitude_cme,latitude_cme,time_cme,flare_id,peak_time_flr,type_flr,latitude_flr,longitude_flr
0,2020-11-20T19:51:00-CME-001,O,1166.0,15.5,-39.0,3.0,2020-11-20T22:48Z,2020-11-20T16:44:00-FLR-001,2020-11-20T17:03Z,C1.9,22,40
1,2020-11-29T13:24:00-CME-001,O,1336.0,58.0,-75.0,-6.0,2020-11-29T16:00Z,2020-11-29T12:34:00-FLR-001,2020-11-29T13:11Z,M4.4,25,97
2,2020-12-07T16:24:00-CME-001,O,1383.0,41.0,12.0,-21.0,2020-12-07T18:49Z,2020-12-07T15:46:00-FLR-001,2020-12-07T16:32Z,C7.4,23,-11
3,2020-12-13T09:24:00-CME-001,C,610.0,26.0,-121.0,3.0,2020-12-13T14:41Z,2020-12-13T08:45:00-FLR-001,2020-12-13T08:55Z,B1.4,18,90
4,2020-12-14T15:12:00-CME-001,S,390.0,19.5,-55.0,-13.2,2020-12-15T00:16Z,2020-12-14T14:09:00-FLR-001,2020-12-14T14:37Z,C4.0,22,44


Os dados relevantes estão selecionados, agora um breve tratamento será realizado.



In [452]:
df = df_final.dropna(subset=['peak_time_flr', 'time_cme'])
#df=df.drop(index=83)

In [453]:
# Conversão das colunas com data e hora para o formato apropriado

rows_to_delete = []
for idx, value in enumerate(df['peak_time_flr']):
    try:
        pd.to_datetime(value)
    except pd._libs.tslib.OutOfBoundsDatetime:
        print(f"Timestamp out of bounds error occurred. Deleting row {idx}.")
        rows_to_delete.append(idx)
df = df.drop(df.index[rows_to_delete])

rows_to_delete2 = []

for idx, value in enumerate(df['time_cme']):
    try:
        pd.to_datetime(value)
    except pd._libs.tslib.OutOfBoundsDatetime:
        print(f"Timestamp out of bounds error occurred. Deleting row {idx}.")
        rows_to_delete2.append(idx)
df = df.drop(df.index[rows_to_delete2])

df['peak_time_flr'] = pd.to_datetime(df['peak_time_flr'])
df['time_cme'] = pd.to_datetime(df['time_cme'])

df['time_difference'] = df['time_cme'] - df['peak_time_flr']

Timestamp out of bounds error occurred. Deleting row 84.


In [454]:
df[0:5]

,cme_id,type_cme,speed,half_angle,longitude_cme,latitude_cme,time_cme,flare_id,peak_time_flr,type_flr,latitude_flr,longitude_flr,time_difference
0,2020-11-20T19:51:00-CME-001,O,1166.0,15.5,-39.0,3.0,2020-11-20 22:48:00+00:00,2020-11-20T16:44:00-FLR-001,2020-11-20 17:03:00+00:00,C1.9,22,40,0 days 05:45:00
1,2020-11-29T13:24:00-CME-001,O,1336.0,58.0,-75.0,-6.0,2020-11-29 16:00:00+00:00,2020-11-29T12:34:00-FLR-001,2020-11-29 13:11:00+00:00,M4.4,25,97,0 days 02:49:00
2,2020-12-07T16:24:00-CME-001,O,1383.0,41.0,12.0,-21.0,2020-12-07 18:49:00+00:00,2020-12-07T15:46:00-FLR-001,2020-12-07 16:32:00+00:00,C7.4,23,-11,0 days 02:17:00
3,2020-12-13T09:24:00-CME-001,C,610.0,26.0,-121.0,3.0,2020-12-13 14:41:00+00:00,2020-12-13T08:45:00-FLR-001,2020-12-13 08:55:00+00:00,B1.4,18,90,0 days 05:46:00
4,2020-12-14T15:12:00-CME-001,S,390.0,19.5,-55.0,-13.2,2020-12-15 00:16:00+00:00,2020-12-14T14:09:00-FLR-001,2020-12-14 14:37:00+00:00,C4.0,22,44,0 days 09:39:00


In [455]:
df.isna().sum()

cme_id             0
type_cme           0
speed              0
half_angle         0
longitude_cme      0
latitude_cme       0
time_cme           0
flare_id           0
peak_time_flr      0
type_flr           0
latitude_flr       0
longitude_flr      0
time_difference    0
dtype: int64

### Matriz de correlação
Para obtermos a matriz de correlação desse banco de dados, é necessário converter todos os dados para o tipo numeric.

In [456]:
# Convertendo os dados para o tipo numeric
# Colunas com numeros
def custom_conversion(column):
    return pd.to_numeric(column, errors='coerce')
cols_numeric = ['speed','half_angle','longitude_cme', 'latitude_cme', 'longitude_flr', 'latitude_flr']
df[cols_numeric] = df[cols_numeric].apply(custom_conversion)

# Removendo colunas que nao serao utilizadas
df = df.drop(['time_cme','peak_time_flr','cme_id','flare_id','type_cme','type_flr'],axis=1)
df=df.dropna()
df[0:5]

# Dados categóricos
## Tinha tentado usar os dados de tipo convertendo eles, mas nao deu certo
#df = pd.get_dummies(df, columns=['type_cme','type_flr'])

# Formato datetime
df['time_difference'] = pd.to_timedelta(df_cat['time_difference'])
df['time_difference'] = df['time_difference'].dt.total_seconds()
df[0:5]

# Matriz de Correlação
correlation_matrix = df.corr()
correlation_matrix

,speed,half_angle,longitude_cme,latitude_cme,latitude_flr,longitude_flr,time_difference
speed,1.000000,0.314337,-0.047198,0.108858,-0.032096,0.063484,-0.823299
half_angle,0.314337,1.000000,0.094431,-0.057938,0.091127,-0.098243,-0.305353
longitude_cme,-0.047198,0.094431,1.000000,-0.037030,-0.009296,-0.962349,0.082986
latitude_cme,0.108858,-0.057938,-0.037030,1.000000,-0.716547,0.045877,-0.076435
latitude_flr,-0.032096,0.091127,-0.009296,-0.716547,1.000000,-0.023450,0.041620
longitude_flr,0.063484,-0.098243,-0.962349,0.045877,-0.023450,1.000000,-0.107409
time_difference,-0.823299,-0.305353,0.082986,-0.076435,0.041620,-0.107409,1.000000


A matriz de correlação permite comprovar o que já se tem de conhecimento nessa área utilizando informações dos últimos três anos disponíveis nestas APIs. Comprova-se a correlação entre a diferença de tempo entre as explosões solares e as CMEs e a velocidade da CME (82,33%), assim como a correlação forte entre região de ocorrência dos dois eventos (as latitudes apresentaram forte correlação entre si, de 71,65%, e mais ainda as longitudes com 96,23%).

## Loading
A etapa de loading foi realizada criando-se um banco de dados relacional utilizando SQLAlchemy, para que os dados do conjunto obtido possam ser utilizados após tratamento em outros projetos.

In [450]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///cme_flr_database.db')
df.to_sql('Correlation Matrix of CME events vs Solar Flare events', con=engine, if_exists='replace', index=False)

245

In [451]:
import sqlite3

conn = sqlite3.connect('cme_flr_database.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM 'Correlation Matrix of CME events vs Solar Flare events'")
results = cursor.fetchall()

cursor.close()
conn.close()

for row in results:
    print(row)


(1166.0, 15.5, -39.0, 3.0, 22, 40, 2.07e-05, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(1336.0, 58.0, -75.0, -6.0, 25, 97, 1.014e-05, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(1383.0, 41.0, 12.0, -21.0, 23, -11, 8.22e-06, 0, 